In [30]:
#usr/bin/env/python3
import torch as __T__
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import *
__T__.manual_seed(1337)

In [8]:
#SVD ON RANK DEFICIENT MATRIX:
__d : int = int(1e3)
__k : int = int(1e3)

__W_rank : int = int(1e2)
W : __T__.cuda.DoubleTensor = __T__.randn(__d, __W_rank) @ __T__.randn(__W_rank, __k)
print(W)
print(f'\n RANK OF MATRIX W : {np.linalg.matrix_rank(W)}')

tensor([[  9.0497,  -3.1722,   6.0432,  ...,  -5.0396,  12.7957, -12.2203],
        [-11.1418,  -5.9670,   7.7003,  ..., -10.3299,  10.8481,  10.5139],
        [-21.7219,   0.4937,  15.5838,  ...,  -4.8988,  -5.0047,  -3.9311],
        ...,
        [  4.0619,  10.8808,   3.4869,  ..., -16.7343,  -2.7286,  16.4703],
        [  2.2962,   2.5495,  -9.2799,  ...,   0.6069,   0.5630,   2.2210],
        [  3.9346,   8.7400,  -2.9693,  ...,  13.3873,  -6.2420,   2.4494]])

 RANK OF MATRIX W : 100


In [9]:
U, S, V = __T__.svd(W)

#RANK r FACTORIZATION 
U_r : __T__.cuda.DoubleTensor = U[:, : __W_rank]
S_r : __T__.cuda.DoubleTensor = __T__.diag(S[: __W_rank])
V_r : __T__.cuda.DoubleTensor = V[:, : __W_rank].t()

B : __T__.cuda.DoubleTensor = U_r @ S_r
A : __T__.cuda.DoubleTensor= V_r
print(f'SHAPE OF B : {B.shape}')
print(f'SHAPE OF A : {A.shape}')

SHAPE OF B : torch.Size([1000, 100])
SHAPE OF A : torch.Size([100, 1000])


In [10]:
bias : __T__.cuda.DoubleTensor = __T__.randn(__d)
x : __T__.cuda.DoubleTensor = __T__.randn(__d)
y_org : __T__.cuda.DoubleTensor = W @ x + bias
y_lord : __T__.cuda.DoubleTensor = (B @ A) @ x + bias
print("ORIGINAL y : \n", y_org)
print("LORDECOMPOSITION : \n", y_lord)
print("PARAMS FOR W : ", W.nelement())
print("PARAMS FOR LORDECOMPOSTION : ", B.nelement() + A.nelement())

ORIGINAL y : 
 tensor([ 5.7144e+01, -8.1995e+01, -1.4738e+02, -1.4767e+01, -7.6083e+02,
         5.6425e+02,  2.4467e+02,  3.1799e+02, -1.0773e+02,  2.9801e+02,
        -4.1974e+02,  1.9744e+02,  1.3310e+02,  6.8457e+01,  1.2593e+02,
         2.4615e+02,  4.6234e+01,  8.6925e+02, -4.0376e+02,  5.1600e+02,
         4.7942e+01, -5.4773e+01, -1.1973e+02, -3.3681e+02, -2.3289e+02,
        -1.1506e+02,  1.1023e+02, -8.6167e+01,  3.5567e+02, -1.7758e+02,
        -1.2409e+02,  3.9873e+02,  2.7278e+02, -2.4280e+01,  1.0292e+02,
        -4.9170e+02,  8.0439e+00, -1.4529e+02,  1.0904e+02,  1.0627e+02,
         1.3102e+02, -3.4549e+01, -1.8561e+02, -8.1543e+01, -3.0084e+02,
         3.3054e+01,  6.4448e+01,  4.8099e+02, -1.3649e+02, -2.3533e+02,
        -3.4932e+02, -1.8377e+02, -4.1827e+01, -3.7975e+02, -2.0687e+02,
        -5.8278e+00, -7.4829e+02, -4.8551e+01,  5.8468e+01, -2.6475e+02,
         1.9714e+02,  1.2015e+02,  1.0542e+02,  2.1464e+02,  6.1223e+01,
         2.6325e+02,  1.9583e+02,  1

In [11]:
#LORA FOR SINGLE DIGIT FINETUNING ON A MNIST CLASSIFICATION TASK 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
mnist_trainset = datasets.MNIST(root = './data', train = True, download = True, transform = transform)
train_loader = __T__.utils.data.DataLoader(mnist_trainset, batch_size = 10, shuffle = True)
mnist_testset = datasets.MNIST(root = './data', train = False, download = True, transform = transform)
test_loader = __T__.utils.data.DataLoader(mnist_testset, batch_size = 10, shuffle = True)
device = __T__.device("cuda:0" if __T__.cuda.is_available() else "cpu")

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34062035.06it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1084609.47it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10100158.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3154061.05it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [13]:
class RichNet(nn.Module):
    def __init__(self, hidden_size_1 : int = 1000, hidden_size_2 : int = 2000):
        super(RichNet,self).__init__()
        self.linear1 = nn.Linear(28 * 28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28 * 28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

net = RichNet().to(device)

In [16]:
def train(train_loader, net, __epochs : int = 5, TOTAL_ITER_LIMIT = None) -> None:
    cross_entropy_loss = nn.CrossEntropyLoss()
    optimizer = __T__.optim.Adam(net.parameters(), lr = 0.001)

    __total_iters : int = 0

    for __epoch in range(__epochs):
        net.train()
        loss_sum : float = 0
        __iter_num : int = 0

        data_iterator = tqdm(train_loader, desc = f'Epoch {__epoch + 1}')
        if TOTAL_ITER_LIMIT is not None: data_iterator.total = TOTAL_ITER_LIMIT
        for data in data_iterator:
            __iter_num += 1
            __total_iters += 1
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28 * 28))
            loss = cross_entropy_loss(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / __iter_num
            data_iterator.set_postfix(loss = avg_loss)
            loss.backward()
            optimizer.step()

            if TOTAL_ITER_LIMIT is not None and __total_iters >= TOTAL_ITER_LIMIT: return

train(train_loader, net, __epochs = 1)

Epoch 1: 100%|██████████| 6000/6000 [00:31<00:00, 188.70it/s, loss=0.124]


In [23]:
original_weights : Dict[str, nn.parameter.Parameter] = {}
for name, param in net.named_parameters(): original_weights[name] = param.clone().detach()

In [24]:
def test():
    correct : int = 0
    total : int = 0
    wrong_counts = [0 for i in range(10)]

    with __T__.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x.view(-1, 784))
            for idx, i in enumerate(output):
                if __T__.argmax(i) == y[idx] : correct += 1
                else : wrong_counts[y[idx]] += 1
                total += 1
    print(f'Accuracy: {round(correct / total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')
test()

Testing: 100%|██████████| 1000/1000 [00:03<00:00, 297.83it/s]

Accuracy: 0.963
wrong counts for the digit 0: 26
wrong counts for the digit 1: 28
wrong counts for the digit 2: 31
wrong counts for the digit 3: 65
wrong counts for the digit 4: 33
wrong counts for the digit 5: 15
wrong counts for the digit 6: 27
wrong counts for the digit 7: 49
wrong counts for the digit 8: 22
wrong counts for the digit 9: 74


In [25]:
# Print the size of the weights matrices of the network
# Save the count of the total number of parameters
TOTAL_ORG_PARAMS : int = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    TOTAL_ORG_PARAMS += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index + 1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total number of parameters: {TOTAL_ORG_PARAMS:,}')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10])
Total number of parameters: 2,807,010


In [38]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank = 1, alpha = 1, device = 'cpu'):
        super().__init__()
        # Section 4.1 of the paper: 
        #   We use a random Gaussian initialization for A and zero for B, so 
        #   ∆W = BA is zero at the beginning of training
        self.lora_A = nn.Parameter(__T__.zeros((rank,features_out)).to(device))
        self.lora_B = nn.Parameter(__T__.zeros((features_in, rank)).to(device))
        nn.init.normal_(self.lora_A, mean = 0, std = 1)
        
        # Section 4.1 of the paper: 
        #   We then scale ∆Wx by α/r , where α is a constant in r. 
        #   When optimizing with Adam, tuning α is roughly the same as tuning the learning 
        #   rate if we scale the initialization appropriately. 
        #   As a result, we simply set α to the first r we try and do not tune it. 
        #   This scaling helps to reduce the need to retune hyperparameters when we vary r.
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, original_weights):
        if self.enabled:
            # Return W + (B*A)*scale
            return original_weights + __T__.matmul(self.lora_B, \
                                                   self.lora_A).view(original_weights.shape) * self.scale
        else: return original_weights

In [39]:
#Adding the parametrization to the net
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank = 1, lora_alpha = 1):
    # Only add the parameterization to the weight matrix, ignore the Bias
    # From section 4.2 of the paper:
    #   We limit our study to only adapting the attention weights for downstream tasks and 
    #   freeze the MLP modules (so they are not trained in downstream tasks) both for simplicity and 
    #   parameter-efficiency.
    
    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank = rank, alpha = lora_alpha, device = device
    )

parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization(net.linear1, device)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization(net.linear2, device)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization(net.linear3, device)
)


def enable_disable_lora(enabled = True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled

In [42]:
TOTAL_PARAMS_LORA : int = 0
TOTAL_PARAMS_NON_LORA : int = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    TOTAL_PARAMS_LORA += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    TOTAL_PARAMS_NON_LORA += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index + 1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# The non-LoRA parameters count must match the original network
assert TOTAL_PARAMS_NON_LORA == TOTAL_ORG_PARAMS
print(f'Total number of parameters (original): {TOTAL_PARAMS_NON_LORA:,}')
print(f'Total number of parameters (original + LoRA): {TOTAL_PARAMS_LORA + TOTAL_PARAMS_NON_LORA:,}')
print(f'Parameters introduced by LoRA: {TOTAL_PARAMS_LORA:,}')
parameters_increment = (TOTAL_PARAMS_LORA / TOTAL_PARAMS_NON_LORA) * 100
print(f'Parameters incremment: {parameters_increment:.3f}%')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000]) + Lora_A: torch.Size([1, 784]) + Lora_B: torch.Size([1000, 1])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000]) + Lora_A: torch.Size([1, 1000]) + Lora_B: torch.Size([2000, 1])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10]) + Lora_A: torch.Size([1, 2000]) + Lora_B: torch.Size([10, 1])
Total number of parameters (original): 2,807,010
Total number of parameters (original + LoRA): 2,813,804
Parameters introduced by LoRA: 6,794
Parameters incremment: 0.242%


In [46]:
# Freeze the non-Lora parameters
for name, param in net.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# Load the MNIST dataset again, by keeping only the digit 9
mnist_trainset = datasets.MNIST(root = './data', train = True, download = True, transform = transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# Create a dataloader for the training
train_loader = __T__.utils.data.DataLoader(mnist_trainset, batch_size = 10, shuffle = True)

# Train the network with LoRA only on the digit 9 and only for 100 batches
train(train_loader, net, __epochs = 1, TOTAL_ITER_LIMIT = 100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:  99%|█████████▉| 99/100 [00:00<00:00, 159.24it/s, loss=0.0457]


In [48]:
# Check that the frozen parameters are still unchanged by the finetuning
assert __T__.all(net.linear1.parametrizations.weight.original == original_weights['linear1.weight'])
assert __T__.all(net.linear2.parametrizations.weight.original == original_weights['linear2.weight'])
assert __T__.all(net.linear3.parametrizations.weight.original == original_weights['linear3.weight'])

enable_disable_lora(enabled = True)
# The new linear1.weight is obtained by the "forward" function of our LoRA parametrization
# The original weights have been moved to net.linear1.parametrizations.weight.original
# https://pytorch.org/tutorials/intermediate/parametrizations.html#inspecting-a-parametrized-module
assert __T__.equal(net.linear1.weight, net.linear1.parametrizations.weight.original + (net.linear1.parametrizations.weight[0].lora_B @ net.linear1.parametrizations.weight[0].lora_A) * net.linear1.parametrizations.weight[0].scale)

enable_disable_lora(enabled = False)
# If we disable LoRA, the linear1.weight is the original one
assert __T__.equal(net.linear1.weight, original_weights['linear1.weight'])

In [49]:
# Test with LoRA enabled
enable_disable_lora(enabled = True)
test()

Testing: 100%|██████████| 1000/1000 [00:03<00:00, 263.64it/s]

Accuracy: 0.925
wrong counts for the digit 0: 46
wrong counts for the digit 1: 78
wrong counts for the digit 2: 48
wrong counts for the digit 3: 112
wrong counts for the digit 4: 114
wrong counts for the digit 5: 45
wrong counts for the digit 6: 57
wrong counts for the digit 7: 85
wrong counts for the digit 8: 158
wrong counts for the digit 9: 8


In [50]:
# Test with LoRA disabled
enable_disable_lora(enabled = False)
test()

Testing: 100%|██████████| 1000/1000 [00:03<00:00, 287.17it/s]

Accuracy: 0.963
wrong counts for the digit 0: 26
wrong counts for the digit 1: 28
wrong counts for the digit 2: 31
wrong counts for the digit 3: 65
wrong counts for the digit 4: 33
wrong counts for the digit 5: 15
wrong counts for the digit 6: 27
wrong counts for the digit 7: 49
wrong counts for the digit 8: 22
wrong counts for the digit 9: 74
